# Exploring two dimer toy model with (symmetric) RBM

In [2]:
import netket as nk
import numpy as np
import time
import json
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import jax
import flax
import optax
import pprint
print("NetKet version: {}".format(nk.__version__))
print("NumPy version: {}".format(np.__version__))

NetKet version: 3.3.1
NumPy version: 1.20.3


Setup relevant parameters

In [3]:
"""lattice"""
SITES    = 4             # 4, 8, 16, 20 ... number of vertices in a tile determines the tile shape 
JEXCH1   = .1            # nn interaction
JEXCH2   = 1            # nnn interaction
#USE_MSR  = True        # should we use a Marshall sign rule?
"""machine learning"""
MACHINE = "RBM"
TOTAL_SZ = None            # 0, None ... restriction of Hilbert space
DTYPE = np.complex128      #np.float64 #double #np.complex128   # type of weights in neural network
SAMPLER = 'exact'       # 'local' = MetropolisLocal, 'exchange' = MetropolisExchange
ALPHA = 16               # N_hidden / N_visible
ETA   = .005             # learning rate (0.01 usually works)
SAMPLES = 1000
NUM_ITER = 500
num_layers = 2 #2          # number of layers in G-CNN
feature_dims = (8,4) #(8,4) #(8,8,8,8) # dimensions of layers in G-CNN

OUT_NAME = "SS-RBM_ops"+str(SITES)+"j1="+str(JEXCH1) # output file name

Lattice and hamiltonian definition: &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; $ H = J_{1} \sum\limits_{\langle i,j \rangle}^{L} \vec{\sigma}_{i} \cdot \vec{\sigma}_{j} + J_{2} \sum\limits_{\langle\langle i,j \rangle\rangle_{SS}}^{L}  \vec{\sigma}_{i} \cdot \vec{\sigma}_{j}\,, $

In [4]:
# Define custom graph
# edge_colors = []

# edge_colors.append([0,1,1])
# edge_colors.append([2,3,1])
# # # SS 2x2 lattice:
# edge_colors.append([1,2,1])
# edge_colors.append([0,3,1])
# edge_colors.append([0,2,2])
# edge_colors.append([1,3,2])
# # double bonds
# edge_colors.append([0,1,1])
# edge_colors.append([2,3,1])
# edge_colors.append([1,2,1])
# edge_colors.append([0,3,1])

# 3-chain:
# edge_colors.append([0,1,1])
# edge_colors.append([1,2,1])
# edge_colors.append([2,0,1])

from lattice_and_ops import Lattice
lattice = Lattice(SITES)

# Define custom graph
edge_colors = []
for node in range(SITES):
    edge_colors.append([node,lattice.rt(node), 1]) #horizontal connections
    edge_colors.append([node,lattice.bot(node), 1]) #vertical connections
    row, column = lattice.position(node)
    if column%2 == 0:
        if row%2 == 0:
            edge_colors.append([node,lattice.lrt(node),2])
        else:
            edge_colors.append([node,lattice.llft(node),2])

# Define the netket graph object
g = nk.graph.Graph(edges=edge_colors) #,n_nodes=3)
N = g.n_nodes

hilbert = nk.hilbert.Spin(s=.5, N=g.n_nodes, total_sz=TOTAL_SZ)


#Sigma^z*Sigma^z interactions
sigmaz = [[1, 0], [0, -1]]
mszsz = (np.kron(sigmaz, sigmaz)) #=sz*sz
#Exchange interactions
exchange = np.asarray([[0, 0, 0, 0], [0, 0, 2, 0], [0, 2, 0, 0], [0, 0, 0, 0]]) #=sx*sx+sy*sy = 1/2*(sp*sm+sm*sp)
full_spin = mszsz+exchange # = S*S = sx*sx + sy*sy + sz*sz
bond_color = [1, 2, 1, 2]

In [62]:
# translations = []
# for perm in g.automorphisms():
#     aperm = np.asarray(perm)
#     if True:
#     # if (aperm[0],aperm[1]) in ((0,1),(1,0),(2,3),(3,2)): # translations for 2*2 SS model
#     # if (aperm[2],aperm[0]) in ((2,0),(3,0),(0,1),(1,2)):#,(2,3,1)): # N = 16, two dimmers with just translation
#     # if (aperm[0],aperm[1],aperm[3]) in ((0,1,3),(3,2,0),(2,3,1),(1,0,2)): # N = 16, two dimmers plus second translation option
#         translations.append(nk.utils.group._permutation_group.Permutation(aperm))
# # print(translations)
# translation_group = nk.utils.group._permutation_group.PermutationGroup(translations,degree=N)
# # characters = np.array([1,-1,-1,1,1,-1,-1,1],dtype=complex) # in case of just two dimers

In [5]:

print("There are", len(g.automorphisms()), "full symmetries:")
if JEXCH1 < 0.5:
    characters = []
    from lattice_and_ops import permutation_sign
    for perm in g.automorphisms():
        print(perm, permutation_sign(np.asarray(perm)))
        characters.append(permutation_sign(np.asarray(perm)))
    characters_dimer = np.asarray(characters,dtype=complex) #np.array([1,-1,1,-1,-1,1,-1,1], dtype=complex) # 2*2 SS model
    characters_dimer_msr = characters_dimer
else:
    characters_dimer = np.ones((len(g.automorphisms()),), dtype=complex)
    characters_dimer_msr = characters_dimer
print(characters_dimer)
print(characters_dimer_msr)
# print("There are", len(translation_group),"restricted symmetries:")
# print(translation_group)

There are 8 full symmetries:
Permutation([0, 1, 2, 3]) 1
Permutation([0, 2, 1, 3]) -1
Permutation([1, 0, 3, 2]) 1
Permutation([1, 3, 0, 2]) -1
Permutation([2, 0, 3, 1]) -1
Permutation([2, 3, 0, 1]) 1
Permutation([3, 1, 2, 0]) -1
Permutation([3, 2, 1, 0]) 1
[ 1.+0.j -1.+0.j  1.+0.j -1.+0.j -1.+0.j  1.+0.j -1.+0.j  1.+0.j]
[ 1.+0.j -1.+0.j  1.+0.j -1.+0.j -1.+0.j  1.+0.j -1.+0.j  1.+0.j]


In [6]:
translations = []
for perm in g.automorphisms():
    aperm = np.asarray(perm)
    if (aperm[0],aperm[1]) in ((0,1),(1,0),(2,3),(3,2)):
    # if (aperm[0],aperm[1],aperm[3]) in ((0,1,3),):#(2,3,1),(8,9,11),(10,11,9)): # N = 16
    # if (aperm[0],aperm[1],aperm[2],aperm[3]) in ((4,7,6,5),): # N = 8
        translations.append(nk.utils.group._permutation_group.Permutation(aperm))
# print(translations)
translation_group = nk.utils.group._permutation_group.PermutationGroup(translations,degree=SITES)
print(translation_group)
print(len(g.automorphisms()))
for perm in g.automorphisms():
    print(perm)

PermutationGroup(elems=[Permutation([0, 1, 2, 3]), Permutation([1, 0, 3, 2]), Permutation([2, 3, 0, 1]), Permutation([3, 2, 1, 0])], degree=4)
8
Permutation([0, 1, 2, 3])
Permutation([0, 2, 1, 3])
Permutation([1, 0, 3, 2])
Permutation([1, 3, 0, 2])
Permutation([2, 0, 3, 1])
Permutation([2, 3, 0, 1])
Permutation([3, 1, 2, 0])
Permutation([3, 2, 1, 0])


## Hamoltonian

In [7]:
from lattice_and_ops import HamOps
ho = HamOps()
ha = nk.operator.GraphOperator(hilbert, graph=g, bond_ops=ho.bond_operator(JEXCH1,JEXCH2, use_MSR=False), bond_ops_colors=ho.bond_color)
ha_MSR = nk.operator.GraphOperator(hilbert, graph=g, bond_ops=ho.bond_operator(JEXCH1,JEXCH2, use_MSR=True), bond_ops_colors=ho.bond_color)


In [66]:
# # Quick different definition using Hypercube & Heisenberg
# L = 4
# g = nk.graph.Hypercube(length=L, n_dim=2, pbc=True)
# hilbert = nk.hilbert.Spin(s=0.5, total_sz=TOTAL_SZ, N=g.n_nodes)
# ha = nk.operator.Heisenberg(hilbert=hilbert, graph=g, sign_rule=False)
# ha_MSR = nk.operator.Heisenberg(hilbert=hilbert, graph=g, sign_rule=True) 

## Exact diagonalization

In [8]:
if g.n_nodes < 20:
    start = time.time()
    if g.n_nodes < 15:
        evals, eigvects = nk.exact.full_ed(ha, compute_eigenvectors=True)
    else:
        evals, eigvects = nk.exact.lanczos_ed(ha, k=3, compute_eigenvectors=True)
    end = time.time()
    diag_time = end - start
    print("Ground state energy:",evals[0], "\nIt took ", round(diag_time,2), "s =", round((diag_time)/60,2),"min")
else:
    print("System is too large for exact diagonalization. Setting exact_ground_energy = 0 (which is wrong)")
    evals = [0,0,0]
    eigvects = None 
exact_ground_energy = evals[0]
# -36.2460684609957 

Ground state energy: -6.000000000000002 
It took  2.97 s = 0.05 min


In [68]:
print(evals)
# print([eigvects[i,0] if np.abs(eigvects[i,0]) > 0.001 else 0 for i in range(len(eigvects[:,0]))])

[-6.  -2.  -2.  -2.  -2.  -2.  -2.   0.4  1.2  1.2  1.2  2.8  2.8  2.8
  2.8  2.8]


In [69]:
from scipy.sparse.linalg import eigsh

In [70]:
evals, eigvects = eigsh(ha_MSR.to_sparse())

In [71]:
(eigvects[:,0] > 0.00001) + (eigvects[:,0] < -0.00001)

array([False, False, False,  True, False,  True, False, False, False,
       False,  True, False,  True, False, False, False])

In [72]:
arr = (eigvects[:,0] > 0.00001) + (eigvects[:,0] < -0.00001)
for i,j in enumerate(arr):
    if j:
        print(i,eigvects[i,0])

3 0.5000000000000002
5 -0.5
10 -0.5
12 0.4999999999999999


## RBM
```

In [21]:
if MACHINE == "ModPhase":
    # A linear schedule varies the learning rate from 0 to 0.01 across 600 steps.
    modulus_schedule=optax.linear_schedule(0,0.01,NUM_ITER)
    modulus_schedule_MSR=optax.linear_schedule(0,0.01,NUM_ITER)

    # The phase starts with a larger learning rate and then is decreased.
    phase_schedule=optax.linear_schedule(0.05,0.01,NUM_ITER)
    phase_schedule_MSR=optax.linear_schedule(0.05,0.01,NUM_ITER)

    # Combine the linear schedule with SGD
    optm=optax.sgd(modulus_schedule)
    optp=optax.sgd(phase_schedule)
    optm_MSR=optax.sgd(modulus_schedule_MSR)
    optp_MSR=optax.sgd(phase_schedule_MSR)

    # The multi-transform optimizer uses different optimisers for different parts of the
    # parameters.
    optimizer = optax.multi_transform({'o1': optm, 'o2': optp}, flax.core.freeze({"Dense_0":"o1", "Dense_1":"o2"}))
    optimizer_MSR = optax.multi_transform({'o1': optm_MSR, 'o2': optp_MSR}, flax.core.freeze({"Dense_0":"o1", "Dense_1":"o2"}))
else:
    optimizer = nk.optimizer.Sgd(learning_rate=ETA)
    optimizer_MSR = nk.optimizer.Sgd(learning_rate=ETA)

In [22]:
#GCNN
#definice modelu, sampleru atd.

#Feature dimensions of hidden layers, from first to last
#feature_dims = (8,8,8,8)

#Number of layers
#num_layers = 4

#Define the GCNN
# machine = nk.models.GCNN(symmetries = g.automorphisms(), layers = num_layers, features = feature_dims)
# machine_MSR = nk.models.GCNN(symmetries = g.automorphisms(), layers = num_layers, features = feature_dims)

In [23]:
#definice modelu, sampleru atd.
if MACHINE == "RBM":
    machine =     nk.models.RBM(dtype=DTYPE, alpha=ALPHA)#, use_visible_bias=False) 
    machine_MSR = nk.models.RBM(dtype=DTYPE, alpha=ALPHA)#, use_visible_bias=False)
elif MACHINE == "RBMModPhase":
    machine = nk.models.RBMModPhase(alpha=ALPHA, use_hidden_bias=True, dtype=DTYPE)
    machine_MSR = nk.models.RBMModPhase(alpha=ALPHA, use_hidden_bias=True, dtype=DTYPE)
elif MACHINE == "GCNN":
    machine     = nk.models.GCNN(symmetries=g.automorphisms(), dtype=DTYPE, layers=num_layers, features=feature_dims, characters=characters_dimer)
    machine_MSR = nk.models.GCNN(symmetries=g.automorphisms(), dtype=DTYPE, layers=num_layers, features=feature_dims, characters=characters_dimer_msr)
elif MACHINE == "RBMSymm":
    machine =     nk.models.RBMSymm(g.automorphisms(), dtype=DTYPE, alpha=ALPHA)#, use_visible_bias=False) 
    machine_MSR = nk.models.RBMSymm(g.automorphisms(), dtype=DTYPE, alpha=ALPHA)#, use_visible_bias=False)
elif MACHINE == "RBMSymm_transl":
    machine =     nk.models.RBMSymm(translation_group, dtype=DTYPE, alpha=ALPHA)#, use_visible_bias=False) 
    machine_MSR = nk.models.RBMSymm(translation_group, dtype=DTYPE, alpha=ALPHA)#, use_visible_bias=False)
else:
    raise Exception(str("undefined MACHINE: ")+str(MACHINE))

# Meropolis Exchange Sampling
if SAMPLER == 'local':
    sampler = nk.sampler.MetropolisLocal(hilbert=hilbert)
    sampler_MSR = nk.sampler.MetropolisLocal(hilbert=hilbert)
elif SAMPLER == 'exact':
    sampler = nk.sampler.ExactSampler(hilbert=hilbert)
    sampler_MSR = nk.sampler.ExactSampler(hilbert=hilbert)
else:
    sampler = nk.sampler.MetropolisExchange(hilbert=hilbert, graph=g)
    sampler_MSR = nk.sampler.MetropolisExchange(hilbert=hilbert, graph=g)
    if SAMPLER != 'exchange':
        print("Warning! Undefined fq.SAMPLER:", SAMPLER, ", dafaulting to MetropolisExchange fq.SAMPLER")


# Stochastic Reconfiguration
sr  = nk.optimizer.SR(diag_shift=0.01)
sr_MSR  = nk.optimizer.SR(diag_shift=0.01)

# The variational state (former name: nk.variational.MCState)
vss = nk.vqs.MCState(sampler, machine, n_samples=SAMPLES)
vs_MSR  = nk.vqs.MCState(sampler_MSR, machine_MSR, n_samples=SAMPLES)
vss.init_parameters(jax.nn.initializers.normal(stddev=0.001))
vs_MSR.init_parameters(jax.nn.initializers.normal(stddev=0.001))


gs_normal = nk.VMC(hamiltonian=ha ,optimizer=optimizer,preconditioner=sr,variational_state=vss)               # 0 ... symmetric
gs_MSR = nk.VMC(hamiltonian=ha_MSR ,optimizer=optimizer_MSR,preconditioner=sr_MSR,variational_state=vs_MSR)   # 1 ... symmetric+MSR

In [24]:
vss.n_parameters

324

# Calculation

In [25]:
no_of_runs = 2 #2 ... bude se pocitat i druhý způsob (za použití MSR)
use_MSR = 0 # in case of one run
# NUM_ITER = 20*3
print("J_1 =", JEXCH1)
if exact_ground_energy != 0:
    print("Expected exact energy:", exact_ground_energy)
for i,gs in enumerate([gs_normal,gs_MSR][use_MSR:use_MSR+no_of_runs]):
    start = time.time()
    gs.run(out=OUT_NAME+str(i), n_iter=int(NUM_ITER))#, obs={'symmetry':P(0,1)})
    end = time.time()
    print("The type {} of RBM calculation took {} min".format(i, (end-start)/60))


J_1 = 0.1
Expected exact energy: -6.000000000000002


  0%|          | 0/150 [00:00<?, ?it/s]

x=Traced<ShapedArray(complex128[64])>with<DynamicJaxprTrace(level=0/1)>  target=Traced<ShapedArray(complex128[64])>with<DynamicJaxprTrace(level=0/1)>
x=Traced<ShapedArray(complex128[4,64])>with<DynamicJaxprTrace(level=0/1)>  target=Traced<ShapedArray(complex128[4,64])>with<DynamicJaxprTrace(level=0/1)>
x=Traced<ShapedArray(complex128[4])>with<DynamicJaxprTrace(level=0/1)>  target=Traced<ShapedArray(complex128[4])>with<DynamicJaxprTrace(level=0/1)>
x=Traced<ShapedArray(complex128[64])>with<DynamicJaxprTrace(level=1/2)>  target=Traced<ShapedArray(complex128[64])>with<DynamicJaxprTrace(level=1/2)>
x=Traced<ShapedArray(complex128[4,64])>with<DynamicJaxprTrace(level=1/2)>  target=Traced<ShapedArray(complex128[4,64])>with<DynamicJaxprTrace(level=1/2)>
x=Traced<ShapedArray(complex128[4])>with<DynamicJaxprTrace(level=1/2)>  target=Traced<ShapedArray(complex128[4])>with<DynamicJaxprTrace(level=1/2)>
x=Traced<ShapedArray(complex128[64])>with<DynamicJaxprTrace(level=1/2)>  target=Traced<ShapedArr

100%|██████████| 150/150 [00:08<00:00, 17.50it/s, Energy=2.7958+0.0003j ± 0.0057 [σ²=0.0324, R̂=0.9991]]   


The type 0 of RBM calculation took 0.27701428333918254 min


100%|██████████| 150/150 [00:09<00:00, 15.79it/s, Energy=0.678-0.000j ± 0.013 [σ²=0.183, R̂=0.9976]]

The type 1 of RBM calculation took 0.15933237075805665 min


In [ ]:
from lattice_and_ops import Operators, Lattice
ops = Operators(lattice,hilbert,mszsz,exchange)
for i,gs in enumerate([gs_normal,gs_MSR][use_MSR:use_MSR+no_of_runs]):
    print("Trained RBM with MSR:" if i else "Trained RBM without MSR:")
    print("m_d^2 =", gs.estimate(ops.m_dimer_op))
    print("m_p =", gs.estimate(ops.m_plaquette_op_MSR))
    print("m_s^2 =", gs.estimate(ops.m_s2_op_MSR))
    print("m_s^2 =", gs.estimate(ops.m_s2_op), "<--- no MSR!!")

Trained RBM without MSR:
m_d^2 = -0.2590+0.0001j ± 0.0043 [σ²=0.0187, R̂=0.9976]
m_p = 0.000-0.003j ± 0.043 [σ²=1.893, R̂=1.0012]
m_s^2 = 1.0154+0.0012j ± 0.0095 [σ²=0.0904, R̂=0.9970]
m_s^2 = 0.1438+0.0006j ± 0.0031 [σ²=0.0099, R̂=0.9986] <--- no MSR!!
Trained RBM with MSR:
m_d^2 = -3.333e-01+1.774e-08j ± 1.142e-08 [σ²=1.315e-13, R̂=1.0001]
m_p = -0.042+0.000j ± 0.046 [σ²=2.090, R̂=0.9978]
m_s^2 = 1.1278-0.0000j ± 0.0038 [σ²=0.0147, R̂=1.0045]
m_s^2 = 1.250e-01+7.959e-11j ± 9.731e-09 [σ²=9.545e-14, R̂=0.9989] <--- no MSR!!


In [ ]:
print(gs_normal.estimate(2*nk.operator.LocalOperator(hilbert,operators=[[1,0],[0,1]],acting_on=[0])))

2.000e+00+0.000e+00j ± nan [σ²=0.000e+00]


## Energy

In [19]:
threshold_energy = 0.995*exact_ground_energy
data = []
for i in range(no_of_runs):
    data.append(json.load(open(OUT_NAME+str(i)+".log")))
if type(data[0]["Energy"]["Mean"]) == dict: #DTYPE in (np.complex128, np.complex64):#, np.float64):# and False:
    energy_convergence = [data[i]["Energy"]["Mean"]["real"] for i in range(no_of_runs)]
else:
    energy_convergence = [data[i]["Energy"]["Mean"] for i in range(no_of_runs)]

print(np.array(energy_convergence[0]) < threshold_energy)

[False False False False False  True False False False False False  True
 False False False  True False False False False False  True False False
 False False False False False  True False  True False False  True  True
 False  True False False  True  True  True  True  True False False  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True False  True  True  True  True
  True  True  True  True False False  True  True  True  True  True  True
  True  True  True  True False  True  True  True  True False  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]


In [80]:
# exact energy line
figure = go.Figure(
    data=[go.Scatter(x=(0,NUM_ITER),y=(exact_ground_energy,exact_ground_energy),mode="lines",line=go.scatter.Line(color="#000000",width=1), name="exact energy"),go.Scatter(x=(0,NUM_ITER),y=(.995*exact_ground_energy,.995*exact_ground_energy),mode="lines",line=go.scatter.Line(color="#000000",width=1), name="99.5 % of exact energy")], 
    layout=go.Layout(template="simple_white",
        xaxis=dict(title="Iteration", mirror=True, showline=True),
        yaxis=dict(title="Energy", mirror=True, showline=True),
        title=("<b>"+"S-S"+" model </b>, L="+str(SITES)+", J2 ="+str(JEXCH2)+ ", J1 ="+str(JEXCH1)+" , η="+str(ETA)+", α="+str(ALPHA)+", samples="+str(SAMPLES))))

In [85]:
# import the data from log file
OUT_NAME_suffixless=OUT_NAME
data = []
for i in range(no_of_runs):
    data.append(json.load(open(OUT_NAME_suffixless+str(i)+".log")))
names = ["normal basis","MSR basis"]
if type(data[0]["Energy"]["Mean"]) == dict: #DTYPE in (np.complex128, np.complex64):#, np.float64):# and False:
    energy_convergence = [data[i]["Energy"]["Mean"]["real"] for i in range(no_of_runs)]
    # symmetry = [data[i]["symmetry"]["Mean"]["real"] for i in range(no_of_runs-use_MSR)]
else:
    energy_convergence = [data[i]["Energy"]["Mean"] for i in range(no_of_runs)]
    # symmetry = [data[i]["symmetry"]["Mean"] for i in range(no_of_runs-use_MSR)]
for i in range(no_of_runs):
    figure.add_trace(go.Scatter(
        x=data[i]["Energy"]["iters"], y=energy_convergence[i],
        name=names[i]
    ))
    # figure.add_trace(go.Scatter(
    #     x=data[i]["Energy"]["iters"], y=symmetry[i],
    #     name=names[i]+"_swap"
    # ))

#figure.add_hline(y=exact_gs_energy)
figure.update_layout(xaxis_title="Iteration",yaxis_title="Energy")
figure.show()

# Calculating symmetrizations